In [6]:
#importing Pandas and Numpy for data analysis
import pandas as pd
import numpy as np
import csv
#using the Pandas readcsv command to load the entire dataset into a dataframe called Titanic_df
Train_df = pd.read_csv('C:/Users/Mash/Desktop/Kaggle/titanic/train.csv')
Test_df = pd.read_csv('C:/Users/Mash/Desktop/Kaggle/titanic/test.csv')

#Outliers and NULL values detected and handled
# Replace the Age columns with null values by the mean of Age
Mean_Age = np.mean(Train_df['Age'])
Train_df['Age'] = np.where(np.isnan(Train_df['Age']),Mean_Age,Train_df['Age'])

#FEATURE ENGINEERING
# Create a new column to classify Adult/Child
Train_df['Adult_Child']= np.where(Train_df['Age']>14,'A','C')

# Creating a Title column
Train_df['Name']=Train_df['Name'].str.replace(".",',')
def split_Name_for_Title(data):
    return data.split(",")[1]
Train_df['Title'] = Train_df['Name'].apply(split_Name_for_Title)
Train_df['Title'] = str(Train_df['Title'].map(lambda x: x.strip()))
#Creating fare categories by binning
def Grading(element):
    if element >=0 and element<=15:
        return 1 # for 0-15
    elif element >=16 and element<=30:
        return 2 # for 16-30
    elif element >=31 and element<=45:
        return 3 # for 31-45
    elif element >=46 and element<=60:
        return 4 # for 46-60
    else:
        return 5 # 61 and above
        
def Binning_Fare(dfSeries):
    return dfSeries.apply(Grading)

#The Continuous variable Fare is converted into Categorical variable and a new colum called Bucketed Fare is created
Train_df['Bucketed_Fare']=Binning_Fare(Train_df['Fare']) 

#Creating family variable
Train_df['Family']= Train_df['SibSp']+Train_df['Parch']
Train_df['Family']= np.where(Train_df['Family'] !=0,1,0)

# splitting lables and features
y = Train_df['Survived']
X = Train_df.ix[:,Train_df.columns != 'Survived']

# Feature Selection
from sklearn.feature_selection import SelectKBest,chi2
X= pd.get_dummies(X)
features = SelectKBest(chi2,k=10).fit(X,y)
X_train = features.transform(X)
mask= np.nonzero(features.get_support())[0]
only_features = X.columns
chosen = []
print ("Chosen features by algorithm:","\n")
for i in range(len(only_features)):
        if i in mask:
            chosen.append(only_features[i])
print (chosen)
# Choosing the algorithm
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
rfc = RandomForestClassifier()
param_grid = {"max_depth": [2,3,None],
                  "max_features": ["auto","log2",0.9],
                  "min_samples_split": [2,4,6],
                  "min_samples_leaf": [1,10,20],
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"],
                  "n_estimators": [10,15,30],
                  "bootstrap":[True],
                  "oob_score": [True],
                  "n_jobs": [-1],
                  "random_state":[50]}
GS = GridSearchCV(estimator = rfc,param_grid=param_grid,scoring="f1")
GS = GS.fit(X_train,y)
print (GS.best_params_)
clf = GS.best_estimator_

#Incorporating new features into test set
Test_df['Bucketed_Fare']=Binning_Fare(Test_df['Fare']) 
Test_df['Family']= Test_df['SibSp']+Train_df['Parch']
Test_df['Family']= np.where(Test_df['Family'] !=0,1,0)
Test_df['Adult_Child']= np.where(Test_df['Age']>14,'A','C')
Mean_Age = np.mean(Test_df['Age'])
Test_df['Age'] = np.where(np.isnan(Test_df['Age']),Mean_Age,Test_df['Age'])
# print (np.argwhere(np.isnan(X_test)))
Mean_Fare = np.mean(Test_df['Fare'])
Test_df['Fare'] = np.where(np.isnan(Test_df['Fare']),0,Test_df['Fare'])
Test_df = pd.get_dummies(Test_df)
X_test = Test_df[chosen].as_matrix()
y_predict = clf.predict(X_test)
predictions_file = open("C:/Users/Mash/Desktop/Kaggle/titanic/RFoutput.csv", "w",newline='')
predictions_file_object = csv.writer(predictions_file)
predictions_file_object.writerow(["PassengerId", "Survived"])
for i in range(len(Test_df)):
    predictions_file_object.writerow([Test_df['PassengerId'][i],y_predict[i]])
predictions_file.close()

Chosen features by algorithm: 

['Pclass', 'Age', 'Parch', 'Fare', 'Bucketed_Fare', 'Family', 'Sex_female', 'Sex_male', 'Embarked_C', 'Adult_Child_C']


C:\Users\Mash\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Mash\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Mash\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Mash\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB sc

{'min_samples_leaf': 1, 'oob_score': True, 'max_features': 0.9, 'min_samples_split': 6, 'bootstrap': True, 'criterion': 'gini', 'n_estimators': 15, 'random_state': 50, 'max_depth': None, 'n_jobs': -1}
